In [10]:
#IMPORT STUFF RELATED TO DATA MANAGEMENT
import pandas as pd
import numpy as np
import sqlite3 as sql
import matplotlib.pyplot as plt
import seaborn as sns
import re

limit = 'limit '+ str(100000)

In [11]:
database_path = 'NEST_Pandemia.sqlite'

conn = sql.connect(database_path) # nest con filtro de pandemia
# conn = sql.connect('NEST_Pandemia.sqlite') # nest sin filtro de pandemia

# extrayendo placa y el año de la fecha de inicio de vigencia de la poliza
# este analisis es por placa, si se quisiera ver persistencia del cliente solo tendríamos que cambiar "numplaca" por "codcli"
query = "SELECT SEXO, CODCLI, CAST((julianday(FECINIVALID) - julianday(FECNAC)) / 365.25 AS INTEGER) AS EDAD, strftime('%Y', FECINIVALID ) ano from nest where tipoveh= 'P' and CODPOL in ('02B','02BR') and FECINIVALID > '2003-01-01' and FECINIVALID < '2024-01-01'  group by CODCLI, ano ;"
# query = "select SEXO, FECNAC,  CODCLI, strftime('%Y', FECINIVALID ) ano from nest where tipoveh= 'P' and FECINIVALID > '2010-01-01' group by CODCLI, ano, SEXO, EDAD " 
#query = "select * from NEST limit 1"
basico = pd.read_sql_query(query, conn)

In [12]:
basico.tail(50)

,SEXO,CODCLI,EDAD,ano
873384,M,00000004075405,29.0,2023
873385,M,00000004075409,34.0,2023
873386,F,00000004075411,55.0,2023
873387,M,00000004075412,68.0,2023
873388,M,00000004075414,36.0,2023
873389,M,00000004075419,27.0,2010
873390,F,00000004075422,38.0,2023
873391,F,00000004075425,29.0,2023
873392,F,00000004075429,60.0,2023
873393,M,00000004075430,25.0,2023


In [13]:
# creando binary table para crear el triangulo de persistencia

basico = basico.sort_values(by=['CODCLI', 'ano'], ascending=[True, True])
#basico['ano'] = pd.to_numeric(basico['ano'], errors='coerce') # Convert the 'ano' column to numeric (integer or float) data type
basico['ano'] = basico['ano'].astype(int)
basico['DETECTOR CORTE PLACA'] = (basico['CODCLI'] == basico['CODCLI'].shift()).astype(int)
basico['DETECTOR CORTE AÑO'] = (basico['ano'].diff() == 1).astype(int)
basico['COMBINADOR DE CORTES'] = (basico['DETECTOR CORTE PLACA'] + basico['DETECTOR CORTE AÑO'] == 2).astype(int)
basico.reset_index(drop=True, inplace=True)
basico.at[0, 'ACUMULADOR'] = 0

# Iterate through the 'basico' DataFrame to calculate cumulative values.

for i in range(len(basico)):
    if i == 0:
        pass
    else:
        if basico['COMBINADOR DE CORTES'][i] == 1:
            value = basico['COMBINADOR DE CORTES'][i] + basico['ACUMULADOR'][i-1]
        else: 
            value = 0
        basico.at[i, 'ACUMULADOR'] = value

basico['ACUMULADOR'] = basico['ACUMULADOR'].astype(int)

# creando binary table

#basico['ACUMULADOR'] = basico.apply(lambda row: row['COMBINADOR DE CORTES'] + row['ACUMULADOR'].shift() if row['COMBINADOR DE CORTES'] == 1 else 0, axis=1)
#basico['ACUMULADOR'] = basico['COMBINADOR DE CORTES'].eq(1).mul(basico['COMBINADOR DE CORTES'] + basico['ACUMULADOR'].shift(fill_value=0))
basico['PRESENTACION'] = basico['ACUMULADOR'] + 1
# Assuming your DataFrame is named 'basico', and the columns are 'AÑO INICIAL' (H), 'ano' (B), 'PRESENTACION' (G), and 'ACUMULADOR' (F)
basico['AÑO INICIAL'] = basico.apply(lambda row: row['ano'] if row['PRESENTACION'] == 1 else row['ano'] - row['ACUMULADOR'], axis=1)

In [14]:
basico["sexo_edad"] = basico["SEXO"] + "_" + basico["EDAD"].astype(str)

In [15]:
basico.to_csv('basicocodcli.csv', index=False)

In [16]:
# hacer pivot table
triangulo = basico.pivot_table(values="CODCLI", index="AÑO INICIAL", columns="ACUMULADOR", aggfunc="count")
#triangulo.reset_index(drop=True)
triangulo = pd.DataFrame(triangulo.to_records())
triangulo

,AÑO INICIAL,0,1,2,3,4,5,6,7,8,...,11,12,13,14,15,16,17,18,19,20
0,2003,11246.0,8576.0,6675.0,5399.0,4492.0,3860.0,3430.0,3094.0,2748.0,...,2005.0,1825.0,1662.0,1508.0,1375.0,1278.0,1180.0,1098.0,1021.0,607.0
1,2004,3228.0,2617.0,2045.0,1566.0,1291.0,1096.0,927.0,766.0,647.0,...,420.0,368.0,325.0,291.0,260.0,237.0,222.0,209.0,125.0,NaN
2,2005,2953.0,2402.0,1814.0,1504.0,1291.0,1086.0,915.0,754.0,648.0,...,434.0,387.0,347.0,318.0,282.0,256.0,227.0,134.0,NaN,NaN
3,2006,2962.0,2477.0,1993.0,1692.0,1439.0,1212.0,1050.0,904.0,779.0,...,498.0,451.0,420.0,377.0,344.0,316.0,169.0,NaN,NaN,NaN
4,2007,6842.0,5199.0,4012.0,3269.0,2680.0,2240.0,1882.0,1590.0,1314.0,...,891.0,803.0,707.0,640.0,573.0,334.0,NaN,NaN,NaN,NaN
5,2008,5515.0,4725.0,3496.0,2827.0,2344.0,1969.0,1641.0,1363.0,1188.0,...,812.0,728.0,657.0,601.0,331.0,NaN,NaN,NaN,NaN,NaN
6,2009,4711.0,3908.0,2862.0,2312.0,1933.0,1639.0,1342.0,1143.0,1010.0,...,681.0,623.0,573.0,312.0,NaN,NaN,NaN,NaN,NaN,NaN
7,2010,14896.0,6284.0,2759.0,2212.0,1815.0,1489.0,1258.0,1100.0,977.0,...,684.0,606.0,306.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2011,13658.0,4219.0,3167.0,2512.0,1999.0,1692.0,1424.0,1236.0,1078.0,...,787.0,410.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2012,6966.0,5822.0,4284.0,3105.0,2468.0,2070.0,1804.0,1588.0,1363.0,...,546.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:

# hacer pivot table
triangulo = basico.pivot_table(values="CODCLI", index="AÑO INICIAL", columns="ACUMULADOR", aggfunc="count")
#triangulo.reset_index(drop=True)
triangulo = pd.DataFrame(triangulo.to_records())

In [18]:
# calcular probabilidades

columnas = triangulo.columns.tolist()[1:]
P_triangulo = pd.DataFrame(triangulo['AÑO INICIAL'])

for i in range(len(columnas)-1):  # Realizar 10 iteraciones para manejar todas las divisiones
    P_triangulo[str(i)] =  triangulo[str(i + 1)] / triangulo[str(i)] 
P_triangulo


# HASTA AQUÍ 

,AÑO INICIAL,0,1,2,3,4,5,6,7,8,...,10,11,12,13,14,15,16,17,18,19
0,2003,0.762582,0.778335,0.808839,0.832006,0.859305,0.888601,0.902041,0.888171,0.885007,...,0.913440,0.910224,0.910685,0.907341,0.911804,0.929455,0.923318,0.930508,0.929872,0.594515
1,2004,0.810719,0.781429,0.765770,0.824393,0.848954,0.845803,0.826321,0.844648,0.863988,...,0.853659,0.876190,0.883152,0.895385,0.893471,0.911538,0.936709,0.941441,0.598086,NaN
2,2005,0.813410,0.755204,0.829107,0.858378,0.841208,0.842541,0.824044,0.859416,0.858025,...,0.871486,0.891705,0.896641,0.916427,0.886792,0.907801,0.886719,0.590308,NaN,NaN
3,2006,0.836259,0.804602,0.848971,0.850473,0.842252,0.866337,0.860952,0.861726,0.834403,...,0.884547,0.905622,0.931264,0.897619,0.912467,0.918605,0.534810,NaN,NaN,NaN
4,2007,0.759866,0.771687,0.814806,0.819823,0.835821,0.840179,0.844846,0.826415,0.891933,...,0.883053,0.901235,0.880448,0.905233,0.895312,0.582897,NaN,NaN,NaN,NaN
5,2008,0.856754,0.739894,0.808638,0.829148,0.840017,0.833418,0.830591,0.871607,0.870370,...,0.889376,0.896552,0.902473,0.914764,0.550749,NaN,NaN,NaN,NaN,NaN
6,2009,0.829548,0.732344,0.807827,0.836073,0.847905,0.818792,0.851714,0.883640,0.869307,...,0.875321,0.914831,0.919743,0.544503,NaN,NaN,NaN,NaN,NaN,NaN
7,2010,0.421858,0.439052,0.801740,0.820524,0.820386,0.844862,0.874404,0.888182,0.892528,...,0.897638,0.885965,0.504950,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2011,0.308903,0.750652,0.793180,0.795780,0.846423,0.841608,0.867978,0.872168,0.884972,...,0.902523,0.520966,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2012,0.835774,0.735830,0.724790,0.794847,0.838736,0.871498,0.880266,0.858312,0.880411,...,0.517536,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
